In [47]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from collections import Counter

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [49]:
def label_map(label):
    if label == "neutral":
        return 0
    elif label == "anger":
        return 1
    elif label == "joy":
        return 2
    elif label == "surprise":
        return 3
    elif label == "sadness":
        return 4
    elif label == "disgust":
        return 5
    elif label == "fear":
        return 6
    
def encode(text, word2index, label, N):
    tokenized = word_tokenize(text)
    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return (encoded,label)

def encode_test(text, word2index, N):
    tokenized = word_tokenize(text)
    for i,word in enumerate(tokenized):
        if word2index.get(word)==None:
            tokenized[i]='unk'

    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]

    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    for batch in iterator:
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [50]:
def cross_validation(sample_size, k=5):
    # calculate the size of each fold
    fold_size = np.ones(k , dtype=int) * (sample_size // k) 
    fold_size[:sample_size % k] += 1 # if the sample size is not divisible by k
    # shuffle
    indexs = np.arange(sample_size)
    np.random.default_rng(seed=10).shuffle(indexs)
    # split indexs  
    folds = np.array_split(indexs, k)
    folds = np.array(folds)
    # return training index and val index in each fold
    ret = []
    for i in range(k):
        train_fold_ind = np.delete(np.arange(k), i)
        train_fold = np.concatenate((folds[train_fold_ind]), axis=None) 
        val_fold = folds[i]
        ret.append([train_fold, val_fold])
    return ret

In [51]:
train_df = pd.read_csv('train_HW2dataset.csv')
dev_df = pd.read_csv('dev_HW2dataset.csv')

train_df = train_df[['Emotion','Utterance']]
dev_df = dev_df[['Emotion','Utterance']]

train_set = list(train_df.to_records(index=False))
dev_set = list(dev_df.to_records(index=False))

counts = Counter()
for ds in [train_set, dev_set]:
    for label,text in ds:
        counts.update(word_tokenize(text))

word2index = {'unk':0}
for i,word in enumerate(counts.keys()):
    word2index[word] = i+1
index2word = {v:k for k,v in word2index.items()}

train_set = train_set + dev_set
train_encoded = [(encode(Utterance,word2index,label_map(label),12)) for label, Utterance in train_set]

x = np.array([tweet for tweet, label in train_encoded])
y = np.array([label for tweet, label in train_encoded])

batch_size = 32


In [52]:
src_vocab_size = len(word2index)      
dimension_model = 32                             
num_layers = 5                       
hidden_size = 30                         
linear_hidden_size = 10               
classes = 7                          
dropout = 0.2                            
lr = 1e-3      

In [53]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.embed = torch.nn.Embedding(src_vocab_size, dimension_model)                        
        self.lstm = torch.nn.LSTM(input_size=dimension_model, hidden_size=hidden_size,num_layers=num_layers,dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, linear_hidden_size)
        self.linear1 = torch.nn.Linear(linear_hidden_size, classes)
    def forward(self,data):
        x = self.embed(data)                          
        x,(h_n, c_n) = self.lstm(x.transpose(0, 1))                                       
        x = self.linear(x[-1])                          
        x = self.linear1(x)                            
        return x  

In [54]:
def cross_validation(sample_size, k=5):
    # calculate the size of each fold
    fold_size = np.ones(k , dtype=int) * (sample_size // k) 
    fold_size[:sample_size % k] += 1 # if the sample size is not divisible by k
    # shuffle
    indexs = np.arange(sample_size)
    np.random.default_rng(seed=10).shuffle(indexs)
    # split indexs  
    folds = np.array_split(indexs, k)
    folds = np.array(folds)
    # return training index and val index in each fold
    ret = []
    for i in range(k):
        train_fold_ind = np.delete(np.arange(k), i)
        train_fold = np.concatenate((folds[train_fold_ind]), axis=None) 
        val_fold = folds[i]
        ret.append([train_fold, val_fold])
    return ret

In [55]:
model = LSTM().to(device)                           
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 

best_acc = 0
kfold_index = cross_validation(sample_size = x.shape[0], k=10)
for train_index, val_index in kfold_index:
    train_x = x[train_index]
    train_y = y[train_index]
    dev_x = x[val_index]
    dev_y = y[val_index]
    train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    dev_ds = TensorDataset(torch.from_numpy(dev_x), torch.from_numpy(dev_y))

    train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
    dev_dl = DataLoader(dev_ds, shuffle=True, batch_size=batch_size, drop_last=True)

    for epoch in range(5):
        train_loss, train_acc = train(model, train_dl, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, dev_dl, criterion)

        print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc * 100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc * 100:.2f}%')    
        if best_acc <= valid_acc:
            best_acc = valid_acc
            PATH=f"epoch{epoch+1}_val.accuracy{valid_loss:.3f}%.pt"
            torch.save({
                    'epoch': epoch+1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': valid_loss,
                    }, PATH)

c:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


Epoch: 01, Train Loss: 1.532, Train Acc: 46.76%, Val. Loss: 1.413, Val. Acc: 51.63%
Epoch: 02, Train Loss: 1.410, Train Acc: 52.19%, Val. Loss: 1.383, Val. Acc: 52.13%
Epoch: 03, Train Loss: 1.368, Train Acc: 52.67%, Val. Loss: 1.350, Val. Acc: 52.20%
Epoch: 04, Train Loss: 1.320, Train Acc: 53.59%, Val. Loss: 1.328, Val. Acc: 52.56%
Epoch: 05, Train Loss: 1.273, Train Acc: 55.62%, Val. Loss: 1.304, Val. Acc: 55.26%
Epoch: 01, Train Loss: 1.233, Train Acc: 57.31%, Val. Loss: 1.179, Val. Acc: 58.81%
Epoch: 02, Train Loss: 1.187, Train Acc: 58.83%, Val. Loss: 1.198, Val. Acc: 57.95%
Epoch: 03, Train Loss: 1.155, Train Acc: 59.97%, Val. Loss: 1.179, Val. Acc: 60.30%
Epoch: 04, Train Loss: 1.122, Train Acc: 61.13%, Val. Loss: 1.187, Val. Acc: 58.81%
Epoch: 05, Train Loss: 1.089, Train Acc: 62.36%, Val. Loss: 1.172, Val. Acc: 60.37%
Epoch: 01, Train Loss: 1.069, Train Acc: 63.08%, Val. Loss: 1.076, Val. Acc: 64.28%
Epoch: 02, Train Loss: 1.042, Train Acc: 64.13%, Val. Loss: 1.065, Val. Acc:

In [56]:
test_df = pd.read_csv('test_HW2dataset.csv')
test_df = test_df[['Utterance']]
test_set = test_df.values.tolist()
test_encoded=[]
for sentence in test_set:
    test_encoded += [encode_test(Utterance, word2index, 10) for Utterance in sentence]
test_x = np.array(test_encoded)
test_ds = TensorDataset(torch.from_numpy(test_x))
test_dl = DataLoader(test_ds, shuffle=False)
 
model = LSTM().to(device)                               
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
predict=[]
for deta in test_dl:
    text = deta[0].to(device)
    preds = model(text)
    _, pred = torch.max(preds, 1)
    predict.append(pred.item())

In [ ]:
ans = pd.DataFrame(columns=["index","emotion"])
for ind, p in enumerate(predict):
    ans.loc[ind] = [ind,p]
ans.to_csv("version1.csv",index=False)